In [2]:
import wandb
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm.auto import tqdm

c:\Users\user\anaconda3\envs\pproject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
api = wandb.Api()
runs = api.runs("kdch6686-/ETRI-Multimodal")

wandb: Currently logged in as: kdch6686 (kdch6686-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
rows = []

for run in tqdm(runs):
    rows.append({
        "encoder": run.config.get("encoder_type"),
        "fusion": run.config.get("fusion_type"),
        "best_val_f1": run.summary.get("best_metric"),
        "best_val_loss": run.summary.get("best_val_loss"),
        "state": run.state,   # finished / crashed
    })

df = pd.DataFrame(rows)

100%|██████████| 30/30 [00:07<00:00,  3.80it/s]


In [5]:
from IPython.display import display, HTML
def display_df(df:pd.DataFrame, num=None):
    display(HTML(df.iloc[:, :-1].head(num).to_html(index=False)))

display_df(df, 10)

encoder,fusion,best_val_f1,best_val_loss
mlp,concat,0.669694,1.957218
mlp,advanced_mlp,0.658983,0.737157
mlp,modal_gating,0.653971,232.319285
mlp,sigmoid_modal,0.601469,254.343521
mlp,cross_attention,0.571849,25.590085
mlp,moe,0.634345,7.045812
conv,concat,0.649068,0.751451
conv,advanced_mlp,0.666575,0.733540
conv,modal_gating,0.658953,0.737703
conv,sigmoid_modal,0.671426,0.734845


In [6]:
df = df.sort_values("best_val_f1", ascending=False)
display_df(df, 10)

encoder,fusion,best_val_f1,best_val_loss
lstm+attention,sigmoid_modal,0.703989,0.723650
lstm+attention,moe,0.701810,0.706648
lstm+attention,concat,0.695388,0.697260
lstm+attention,cross_attention,0.695036,0.722203
transformer,modal_gating,0.692510,0.690925
lstm,sigmoid_modal,0.689907,0.739758
lstm,cross_attention,0.687606,0.712297
lstm+attention,modal_gating,0.684514,0.717475
lstm,modal_gating,0.683858,0.722092
lstm+attention,advanced_mlp,0.683426,0.694714


In [7]:
df = df.sort_values("best_val_loss")
display_df(df, 10)

encoder,fusion,best_val_f1,best_val_loss
transformer,modal_gating,0.692510,0.690925
lstm+attention,advanced_mlp,0.683426,0.694714
lstm+attention,concat,0.695388,0.697260
lstm+attention,moe,0.701810,0.706648
lstm,cross_attention,0.687606,0.712297
lstm+attention,modal_gating,0.684514,0.717475
lstm,modal_gating,0.683858,0.722092
lstm+attention,cross_attention,0.695036,0.722203
lstm+attention,sigmoid_modal,0.703989,0.723650
conv,moe,0.676191,0.732994


In [ ]:
pivot = df.pivot(
    index="encoder",
    columns="fusion",
    values="best_val_f1"
)
pivot.to_latex(
    "pivot_results.tex",
    index=True,          # ⭐ 핵심
    index_names=False,   # "encoder" 같은 index 이름 숨김
    float_format="%.3f"
)
pivot

fusion,advanced_mlp,concat,cross_attention,modal_gating,moe,sigmoid_modal
encoder,,,,,,
conv,0.666575,0.649068,0.643657,0.658953,0.676191,0.671426
lstm,0.678056,0.677179,0.687606,0.683858,0.667388,0.689907
lstm+attention,0.683426,0.695388,0.695036,0.684514,0.701810,0.703989
mlp,0.658983,0.669694,0.571849,0.653971,0.634345,0.601469
transformer,0.664182,0.658953,0.676704,0.692510,0.658953,0.676769


In [9]:
best_per_encoder = (
    df.sort_values("best_val_f1", ascending=False)
      .groupby("encoder")
      .head(1)
)

display_df(best_per_encoder)

encoder,fusion,best_val_f1,best_val_loss
lstm+attention,sigmoid_modal,0.703989,0.723650
transformer,modal_gating,0.692510,0.690925
lstm,sigmoid_modal,0.689907,0.739758
conv,moe,0.676191,0.732994
mlp,concat,0.669694,1.957218
